In [1]:
#Most of this code is slightly modified copy+paste from:
#https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
#Not suitable for the project, but fine for testing to see everything works.

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms import ToTensor, Lambda

#Hyper-parameters:
epochs = 30 #(number of iterations - each one takes ~20 seconds)
#other parameters that we probably shouldn't touch
learning_rate = 1e-3
batch_size = 64
loss_fn = nn.CrossEntropyLoss()

#Having trouble getting this to work (need to set dataset tensor's device to cuda)
device = 'cpu'
"""
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
    #converts labels to one-hot encoded tensors
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)))
"""

train_data = torchvision.datasets.CIFAR10(
    root='training',
    train=True,
    download=True,
    #converts images to tensors
    transform=ToTensor()
)
test_data = torchvision.datasets.CIFAR10(
    root='testing',
    train=False,
    download=True,
    #converts images to tensors
    transform=ToTensor()
)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*32*32, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
      
model = NeuralNetwork().to(device)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

KeyboardInterrupt: 